In [ ]:
import numpy as np
import skimage
import skimage.measure
import rasterio
import numpy as np
import glob, os
import zipfile
import tempfile
import shutil
from tqdm import tqdm
from scipy.signal import convolve2d


def find_max_diff(file_path):
  with rasterio.open(file_path) as src:
    array = src.read(1)
    transform = src.transform

  DECIMATE = 1
  array = array[::DECIMATE, ::DECIMATE]
    
  #maxes = skimage.measure.block_reduce(array, (5,5), np.max)
  #mins = skimage.measure.block_reduce(array, (5,5), np.min)
  #diffs = maxes- mins
  #argmax_flat = np.argmax(diffs)
  #sobel_vertical = np.array([[-2, -1, 0, 1, 2],
  #                           [-2, -1, 0, 1, 2],
  #                           [-4, -2, 0, 2, 4],
  #                           [-2, -1, 0, 1, 2],
  #                           [-2, -1, 0, 1, 2]])
  sobel_vertical = np.array([[0,0,0],
                             [-1,0,1],
                             [0,0,0]])
  sobel_diag = np.array([[-0.72,0,0],
                          [0,0,0],
                          [0,0,0.72]])
  
  sobel_horizontal = sobel_vertical.T


  vertical_edges = convolve2d(array, sobel_vertical, boundary='symm', mode='same')
  horizontal_edges = convolve2d(array, sobel_horizontal, boundary='symm', mode='same')
  diagonal_edges = convolve2d(array, sobel_diag, boundary='symm', mode='same')
  gradient_magnitude = np.maximum(np.maximum(abs(vertical_edges), abs(horizontal_edges)), abs(diagonal_edges))

  argmax_flat = np.argmax(gradient_magnitude)

  # Convert flat index to 2D index
  row, col = np.unravel_index(argmax_flat, gradient_magnitude.shape)
  lon, lat = rasterio.transform.xy(transform, rows=row*DECIMATE, cols=col*DECIMATE, offset='center')
  #print(geodesic((lat, lon), (lat, lon +1)).kilometers)
  #print(geodesic((lat, lon), (lat + 1, lon)).kilometers)
  #print(array.shape)
  drama_factor = gradient_magnitude[row, col]
  if drama_factor > 500 and abs(lat) < 60: 
    print(f'Found {drama_factor}m diff at {lat}, {lon} in {file_path}')
  src.close()

directory = "/home/batman/Documents/global_topo_nasa/"
all_zips = glob.glob(directory + "*.zip")[:]
from random import shuffle
shuffle(all_zips)
for zip_file in tqdm(glob.glob(directory + "*.zip")[:]):
  try:
    with tempfile.TemporaryDirectory(dir='/tmp') as temp_dir:
      with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
        for tif_file in glob.glob(temp_dir + "/*dem.tif"):
          find_max_diff(tif_file)
  except Exception as e:
      print(e)

In [ ]:
import zipfile
import tempfile
import rasterio
import matplotlib.pyplot as plt


def get_file_japan(lat, lon, get_subtile=False):
    # Calculate the bottom-left corner of the tile
    lat_bl = 5 * (lat // 5)  # Round down to the nearest multiple of 5
    lon_bl = 5 * (lon // 5)  # Round down to the nearest multiple of 5

    # Calculate the top-right corner of the tile
    lat_tr = lat_bl + 5
    lon_tr = lon_bl + 5

    # Format latitude and longitude for the tile name
    lat_bl_str = f"S{int(abs(lat_bl)):03d}" if lat_bl < 0 else f"N{int(lat_bl):03d}"
    lon_bl_str = f"W{int(abs(lon_bl)):03d}" if lon_bl < 0 else f"E{int(lon_bl):03d}"
    lat_tr_str = f"S{int(abs(lat_tr)):03d}" if lat_tr < 0 else f"N{int(lat_tr):03d}"
    lon_tr_str = f"W{int(abs(lon_tr)):03d}" if lon_tr < 0 else f"E{int(lon_tr):03d}"

    # For subtile, the tile is 1-degree by 1-degree
    lat_round = 1 * (lat // 1)  # Round down to the nearest degree
    lon_round = 1 * (lon // 1)  # Round down to the nearest degree

    # Format subtile latitude and longitude
    lat_sub_str = f"N{int(abs(lat_round)):03d}" if lat_round >= 0 else f"S{int(abs(lat_round)):03d}"
    lon_sub_str = f"E{int(abs(lon_round)):03d}" if lon_round >= 0 else f"W{int(abs(lon_round)):03d}"

    # Subtile name with prefix and suffix
    subtile_name = f"ALPSMLC30_{lat_sub_str}{lon_sub_str}_DSM"
    #subtile_name = f"ALPSMLC30_{lat_sub_str}{lon_sub_str}_MSK"
    # Construct the main tile name
    tile_name = f"{lat_bl_str}{lon_bl_str}_{lat_tr_str}{lon_tr_str}"

    return tile_name, subtile_name


def get_file_(lat, lon):
    # Format the latitude and longitude for the tile name
    lat_dir = 'N' if lat >= 0 else 'S'
    lon_dir = 'E' if lon >= 0 else 'W'
    
    # Round down to the nearest degree
    lat = int(abs(lat // 1))
    lon = int(abs(lon // 1))
    
    # Format to two digits for latitude and three for longitude
    lat_str = f"{lat_dir}{lat:02d}"
    lon_str = f"{lon_dir}{lon:03d}"
    
    # Return the formatted tile name
    return f"ASTGTMV003_{lat_str}{lon_str}"




def show_subtile(lat, lon):
    tile_name, subtile_name = get_file_japan(lat, lon)
    #tile_name = get_file_(lat, lon)
    print(tile_name)
    directory = "/home/batman/Documents/global_topo_japan/"
    #directory = "/home/batman/Documents/global_topo_nasa/"
    #directory = "/home/batman/Documents/alos_v4/"

    zip_file = directory + tile_name + '.zip'
    with tempfile.TemporaryDirectory(dir='/tmp') as temp_dir:
        print(zip_file)
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
           # tif_file = f'{temp_dir}/{tile_name}/{subtile_name}.tif'
            print(glob.glob(f'{temp_dir}/*'))
            print(tile_name)
            #tif_file = f'{temp_dir}/{tile_name}_dem.tif'

            tif_file = f'{temp_dir}/{tile_name}/{subtile_name}.tif'

            #tif_file = '/home/batman/Documents/global_topo_shuttle/here.tif'
            with rasterio.open(tif_file) as src:
                # Read the data from the first band
                data = src.read(1)
                print(data[-1])
                #data = (data & 0x03) == 1
                
                # Get the bounds of the raster
                bounds = src.bounds
                
                # Get the affine transformation for the raster
                transform = src.transform
            
                # Calculate the longitude and latitude coordinates for the corners of the raster
                left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
                longitude = [left, right, right, left, left]
                latitude = [bottom, bottom, top, top, bottom]
            
                # Transform corner coordinates to centroid coordinates of pixels
                lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')
            
            # Plotting the data with geographic coordinates
            fig, ax = plt.subplots(figsize=(10, 10))
            cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
            ax.set_title('Raster Data with Geographic Coordinates')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            fig.colorbar(cax, ax=ax, label='Data values')
            plt.show()

show_subtile( 3.58638888888889, -65.38694444444445)

In [ ]:
import mpld3
mpld3.enable_notebook()

In [ ]:
import zipfile
import tempfile
import rasterio
import matplotlib.pyplot as plt

def find_tile(lat, lon):
    # Format the latitude and longitude for the tile name
    lat_dir = 'N' if lat >= 0 else 'S'
    lon_dir = 'E' if lon >= 0 else 'W'
    
    # Round down to the nearest degree
    lat = int(abs(lat // 1))
    lon = int(abs(lon // 1))
    
    # Format to two digits for latitude and three for longitude
    lat_str = f"{lat_dir}{lat:02d}"
    lon_str = f"{lon_dir}{lon:03d}"
    
    # Return the formatted tile name
    return f"ASTGTMV003_{lat_str}{lon_str}"


def show_subtile(lat, lon):
    tile_name = find_tile(lat, lon)
    print(tile_name)
    directory = "/home/batman/Documents/global_topo_nasa/"

    zip_file = directory + tile_name + '.zip'
    with tempfile.TemporaryDirectory(dir='/tmp') as temp_dir:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            print(glob.glob(temp_dir+'/*'))
            tif_file = f'{temp_dir}/{tile_name}_dem.tif'
            print(tif_file)
        
            with rasterio.open(tif_file) as src:
                # Read the data from the first band
                data = src.read(1)
                print(data.dtype)
                print(np.max(data))
                
                # Get the bounds of the raster
                bounds = src.bounds
                
                # Get the affine transformation for the raster
                transform = src.transform
            
                # Calculate the longitude and latitude coordinates for the corners of the raster
                left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
                longitude = [left, right, right, left, left]
                latitude = [bottom, bottom, top, top, bottom]
            
                # Transform corner coordinates to centroid coordinates of pixels
                lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')
            
            # Plotting the data with geographic coordinates
            fig, ax = plt.subplots(figsize=(10, 10))
            cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
            ax.set_title('Raster Data with Geographic Coordinates')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            fig.colorbar(cax, ax=ax, label='Data values')
            plt.show()

# Example use of the function
import mpld3

mpld3.enable_notebook()
show_subtile(-3.7341666666666664, 121.16388888888878)

In [ ]:
from geopy.distance import geodesic

# New York coordinates
new_york = (40.0, 0.0)

# London coordinates
london = (40.0, 1.0)

# Calculate the distance
distance = geodesic(new_york, london).kilometers
print(f"The distance between New York and London is {distance:.2f} kilometers.")

In [ ]:
import rasterio

# Path to your raster file

# Open the raster file
with rasterio.open(file_path) as src:
    # Get the affine transform for the raster
    transform = src.transform

    # Get the dimensions of the raster
    width = src.width
    height = src.height

    # Create arrays to hold the longitude and latitude
    longitude, latitude = np.meshgrid(np.arange(width), np.arange(height))

    # Apply the transform to get geographic coordinates
    lon, lat = rasterio.transform.xy(transform, latitude, longitude, offset='center')

    lon = np.array(lon)
    lat = np.array(lat)

    # lon and lat are now arrays containing the longitude and latitude of each pixel
    print("Longitude array:")
    print(lon.shape)
    print("Latitude array:")
    print(lat.shape)

In [ ]:
plot()

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio.windows import Window

with rasterio.open(file_path) as img:
    data = img.read(window=Window(0, 0, 100000, 100000))
    show(data)

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Open the raster file
file_path = '/home/batman/Documents/ASTGTMV003_N68W029_dem.tif'
with rasterio.open(file_path) as src:
    # Read the data from the first band
    data = src.read(1)
    
    # Get the bounds of the raster
    bounds = src.bounds
    
    # Get the affine transformation for the raster
    transform = src.transform

    # Calculate the longitude and latitude coordinates for the corners of the raster
    left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
    longitude = [left, right, right, left, left]
    latitude = [bottom, bottom, top, top, bottom]

    # Transform corner coordinates to centroid coordinates of pixels
    lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')

# Plotting the data with geographic coordinates
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
ax.set_title('Raster Data with Geographic Coordinates')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
#ylim(16.4, 16.8)
#xlim(17, 17.4)
fig.colorbar(cax, ax=ax, label='Data values')
plt.show()

In [ ]:
print(min(array.flatten()))

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(array.flatten()[::100])